# Advance QUESTIONS :

## Question_11_Solution:

**Q-1.** A Social Media Influencer collected data on Facebook friend requests and used
a supervised algorithm to predict whether a user would accept a friend request or
not. Dataset This is the Dataset You can use this dataset for this question. Note : Use
only Dask and Use MLflow

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# !pip install dask mlflow

In [51]:
# Import Required Libraries:
import os
import numpy as np
import dask.dataframe as dd
from xgboost import XGBClassifier
import dask.array as da
import mlflow
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [52]:
# Provide the path to your CSV file in Google Drive
csv_path = '/content/drive/MyDrive/Colab Notebooks/Q11.csv'

# Read the CSV file into a DataFrame
df = dd.read_csv(csv_path)
# Display the DataFrame
df.head(2)


,source_node,destination_node
0,418825,1507485
1,284001,1154125


In [53]:
# Load Facebook data:
# df = dd.read_csv('friend_requests.csv')

In [54]:
# Split the dataset into features (X) and the target variable (y)
X = df[['source_node', 'destination_node']]

# Generating a target variable where 1 indicates acceptance and 0 indicates non-acceptance
def generate_accepted(df):
    return np.random.choice([0, 1], size=len(df), p=[0.5, 0.5])

X['accepted'] = X.map_partitions(generate_accepted, meta=('accepted', 'int'))

# Convert Dask DataFrame to Pandas DataFrame
X_pd = X.compute()

# Split the data into training and testing sets using scikit-learn's train_test_split
X_train_pd, X_test_pd, y_train, y_test = train_test_split(
    X_pd[['source_node', 'destination_node']],
    X_pd['accepted'],
    test_size=0.2,
    random_state=42
)

# Convert Pandas DataFrame back to Dask DataFrame
X_train = dd.from_pandas(X_train_pd, npartitions=1)
X_test = dd.from_pandas(X_test_pd, npartitions=1)

# Create an XGBClassifier model and fit it on the training data
model = XGBClassifier()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy*100}")

Model Accuracy: 50.55


In [55]:
# Initialize and start an MLflow run to track the experiment
mlflow.start_run()

# Log the parameters, metrics, and model
mlflow.log_params(model.get_params())
mlflow.log_metric("accuracy", accuracy)
mlflow.sklearn.log_model(model, "model")

# End the MLflow run
mlflow.end_run()